In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.profiler import profile, record_function, ProfilerActivity

# Define a simple model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(10, 50)
        self.fc2 = nn.Linear(50, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model, loss function, and optimizer
model = SimpleModel()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Create some random input data
inputs = torch.randn(100, 10)
targets = torch.randn(100, 1)

# Profile the model
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Print the profiling results
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference        10.83%       3.463ms       100.00%      31.980ms      31.980ms             1  
                                           aten::linear         3.63%       1.161ms        31.50%      10.074ms       5.037ms             2  
                                         aten::mse_loss         3.73%       1.194ms        22.05%       7.051ms       7.051ms             1  
                                            aten::addmm        16.25%       5.198ms        20.11%       6.432ms       3.216ms             2  
      

In [2]:
import torch
import torch.nn as nn

# Define a simple model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(10, 50)
        self.fc2 = nn.Linear(50, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model
model = SimpleModel()

# Convert the model to TorchScript
scripted_model = torch.jit.script(model)

# Save the TorchScript model
scripted_model.save("simple_model.pt")

# Load the TorchScript model
loaded_model = torch.jit.load("simple_model.pt")

# Run inference with the loaded model
inputs = torch.randn(100, 10)
outputs = loaded_model(inputs)


tensor([[-5.3740e-02],
        [ 3.0118e-01],
        [-1.5750e-01],
        [-9.5050e-02],
        [ 2.9309e-03],
        [ 7.8783e-02],
        [-4.0936e-02],
        [ 3.6552e-02],
        [ 2.1995e-01],
        [-1.5086e-01],
        [ 3.2749e-01],
        [ 2.7731e-01],
        [ 1.0149e-01],
        [ 3.4689e-01],
        [-1.7531e-01],
        [ 6.1558e-02],
        [ 4.7356e-01],
        [-9.1545e-03],
        [ 5.9004e-01],
        [-2.1180e-01],
        [ 3.7610e-01],
        [ 8.7792e-02],
        [ 6.3370e-01],
        [-2.4760e-01],
        [-1.3201e-01],
        [ 2.0317e-01],
        [ 2.4588e-01],
        [ 3.9532e-01],
        [ 1.0761e-01],
        [-1.1565e-01],
        [-2.5185e-01],
        [-1.6885e-02],
        [ 2.7115e-01],
        [ 2.9114e-01],
        [ 2.4828e-01],
        [ 2.1044e-01],
        [-1.7291e-01],
        [ 3.5096e-01],
        [-4.0652e-01],
        [-1.3003e-01],
        [ 4.2562e-01],
        [ 2.4875e-01],
        [ 3.0447e-02],
        [ 3

```c++
#include <torch/script.h>
#include <iostream>

int main(int argc, const char* argv[]) {
    if (argc != 2) {
        std::cerr << "Usage: example-app <path_to_model>\n";
        return -1;
    }

    torch::jit::script::Module module;
    try {
        module = torch::jit::load(argv[1]);
    } catch (const c10::Error& e) {
        std::cerr << "Error loading model: " << e.what() << std::endl;
        return -1;
    }

    // Create a sample input tensor
    std::vector<torch::jit::IValue> inputs;
    inputs.push_back(torch::randn({100,10}));

    // Execute the model
    torch::Tensor output = module.forward(inputs).toTensor();

    std::cout << "Output:\n" << output << std::endl;

    return 0;
}
```

In [8]:
import torch
import torch.nn as nn
import torch.quantization

# Define a simple model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(10, 50)
        self.fc2 = nn.Linear(50, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model
model = SimpleModel()

# Ensure the backend is supported
torch.backends.quantized.engine = 'qnnpack'

# Prepare the model for quantization
model.qconfig = torch.quantization.get_default_qconfig('qnnpack')
torch.quantization.prepare(model, inplace=True)

# Calibrate the model (e.g., with a few batches of data)
inputs = torch.randn(100, 10)
model(inputs)

# Convert the model to a quantized model
torch.quantization.convert(model, inplace=True)

# Run inference with the quantized model
outputs = model(inputs)
print(outputs)


NotImplementedError: Could not run 'quantized::linear' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::linear' is only available for these backends: [Meta, QuantizedCPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastXPU, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

Meta: registered at /tmp/nix-build-python3.12-torch-2.4.1.drv-0/source/aten/s